In [1]:
from pickle import load 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)
#SK LEARN
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [2]:
df=pd.read_csv('kc_house_data_test_features.csv',index_col=0)

In [3]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [4]:
df.drop(columns=['id','date','sqft_above','sqft_basement','lat','long','sqft_living15','sqft_lot15'],inplace=True)

In [5]:
df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,yr_built,yr_renovated,zipcode
0,4,2.50,2270,11500,1.0,0,0,3,8,1967,0,98034
1,4,2.50,2270,11500,1.0,0,0,3,8,1967,0,98034
2,3,2.50,1470,1779,2.0,0,0,3,8,2005,0,98029
3,3,1.75,1280,16200,1.0,0,0,3,8,1976,0,98077
4,4,2.75,2830,8126,2.0,0,0,3,8,2005,0,98059


In [6]:
df.isnull().sum()

bedrooms        0
bathrooms       0
sqft_living     0
sqft_lot        0
floors          0
waterfront      0
view            0
condition       0
grade           0
yr_built        0
yr_renovated    0
zipcode         0
dtype: int64

In [7]:
df['yr_old'] =  df['yr_built'].map(lambda x: 2016-x)

In [8]:
df['yr_since_reno'] =  df['yr_renovated'].map(lambda x: 2016-x if x > 0 else 0)

In [9]:
df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,yr_built,yr_renovated,zipcode,yr_old,yr_since_reno
0,4,2.50,2270,11500,1.0,0,0,3,8,1967,0,98034,49,0
1,4,2.50,2270,11500,1.0,0,0,3,8,1967,0,98034,49,0
2,3,2.50,1470,1779,2.0,0,0,3,8,2005,0,98029,11,0
3,3,1.75,1280,16200,1.0,0,0,3,8,1976,0,98077,40,0
4,4,2.75,2830,8126,2.0,0,0,3,8,2005,0,98059,11,0


In [10]:
df.drop(columns=['yr_built','yr_renovated'],inplace=True)

In [11]:
df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,zipcode,yr_old,yr_since_reno
0,4,2.50,2270,11500,1.0,0,0,3,8,98034,49,0
1,4,2.50,2270,11500,1.0,0,0,3,8,98034,49,0
2,3,2.50,1470,1779,2.0,0,0,3,8,98029,11,0
3,3,1.75,1280,16200,1.0,0,0,3,8,98077,40,0
4,4,2.75,2830,8126,2.0,0,0,3,8,98059,11,0


In [12]:
zipcode_dummy = pd.get_dummies(df['zipcode'],prefix = 'zip', drop_first=True)
df2 = pd.concat([df, zipcode_dummy], axis = 1)

In [13]:
df2.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,zipcode,yr_old,yr_since_reno,zip_98002,zip_98003,zip_98004,zip_98005,zip_98006,zip_98007,zip_98008,zip_98010,zip_98011,zip_98014,zip_98019,zip_98022,zip_98023,zip_98024,zip_98027,zip_98028,zip_98029,zip_98030,zip_98031,zip_98032,zip_98033,zip_98034,zip_98038,zip_98039,zip_98040,zip_98042,zip_98045,zip_98052,zip_98053,zip_98055,zip_98056,zip_98058,zip_98059,zip_98065,zip_98070,zip_98072,zip_98074,zip_98075,zip_98077,zip_98092,zip_98102,zip_98103,zip_98105,zip_98106,zip_98107,zip_98108,zip_98109,zip_98112,zip_98115,zip_98116,zip_98117,zip_98118,zip_98119,zip_98122,zip_98125,zip_98126,zip_98133,zip_98136,zip_98144,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,4,2.50,2270,11500,1.0,0,0,3,8,98034,49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4,2.50,2270,11500,1.0,0,0,3,8,98034,49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2.50,1470,1779,2.0,0,0,3,8,98029,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1.75,1280,16200,1.0,0,0,3,8,98077,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2.75,2830,8126,2.0,0,0,3,8,98059,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df2.drop(columns='zipcode',inplace=True)

In [15]:
len(df2.columns)

80

In [16]:
df2['bedrooms'].value_counts()

3     1896
4     1415
2      557
5      363
6       43
1       38
7        5
8        2
0        2
10       1
9        1
Name: bedrooms, dtype: int64

In [17]:
df2['bedrooms']=df2['bedrooms'].map(lambda x: x if x < 15 else 15)

In [18]:
cols=list(df2.columns)

In [19]:
len(cols)

80

In [20]:
#load the scaler

scaler=load(open('scaler.pkl','rb'))

In [21]:
transformed_holdout=pd.DataFrame(data=scaler.transform(df2), columns=cols)

In [22]:
transformed_holdout

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,yr_old,yr_since_reno,zip_98002,zip_98003,zip_98004,zip_98005,zip_98006,zip_98007,zip_98008,zip_98010,zip_98011,zip_98014,zip_98019,zip_98022,zip_98023,zip_98024,zip_98027,zip_98028,zip_98029,zip_98030,zip_98031,zip_98032,zip_98033,zip_98034,zip_98038,zip_98039,zip_98040,zip_98042,zip_98045,zip_98052,zip_98053,zip_98055,zip_98056,zip_98058,zip_98059,zip_98065,zip_98070,zip_98072,zip_98074,zip_98075,zip_98077,zip_98092,zip_98102,zip_98103,zip_98105,zip_98106,zip_98107,zip_98108,zip_98109,zip_98112,zip_98115,zip_98116,zip_98117,zip_98118,zip_98119,zip_98122,zip_98125,zip_98126,zip_98133,zip_98136,zip_98144,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,0.748320,0.652129,0.410520,-0.076500,-0.873621,-0.053629,-0.264139,-0.625611,0.490886,0.124229,-0.152266,-0.10128,-0.121273,-0.078989,-0.087935,-0.136795,-0.085669,-0.113117,-0.069489,-0.097704,-0.080952,-0.096893,-0.111702,-0.16327,-0.059817,-0.141142,-0.117262,-0.11995,-0.111346,-0.114861,-0.077989,-0.13262,6.087610,-0.167475,-0.01969,-0.085209,-0.16352,-0.103215,-0.168451,-0.137967,-0.115896,-0.14785,-0.151905,-0.149755,-0.125481,-0.075434,-0.11658,-0.144252,-0.12675,-0.098108,-0.136501,-0.061744,-0.168936,-0.087486,-0.132317,-0.114166,-0.09851,-0.064225,-0.087035,-0.165013,-0.122582,-0.162267,-0.158196,-0.086582,-0.119283,-0.139995,-0.131099,-0.159988,-0.112412,-0.121273,-0.121273,-0.051402,-0.146476,-0.106984,-0.119617,-0.114514,-0.116922,-0.081917,-0.118276,-0.119617
1,0.748320,0.652129,0.410520,-0.076500,-0.873621,-0.053629,-0.264139,-0.625611,0.490886,0.124229,-0.152266,-0.10128,-0.121273,-0.078989,-0.087935,-0.136795,-0.085669,-0.113117,-0.069489,-0.097704,-0.080952,-0.096893,-0.111702,-0.16327,-0.059817,-0.141142,-0.117262,-0.11995,-0.111346,-0.114861,-0.077989,-0.13262,6.087610,-0.167475,-0.01969,-0.085209,-0.16352,-0.103215,-0.168451,-0.137967,-0.115896,-0.14785,-0.151905,-0.149755,-0.125481,-0.075434,-0.11658,-0.144252,-0.12675,-0.098108,-0.136501,-0.061744,-0.168936,-0.087486,-0.132317,-0.114166,-0.09851,-0.064225,-0.087035,-0.165013,-0.122582,-0.162267,-0.158196,-0.086582,-0.119283,-0.139995,-0.131099,-0.159988,-0.112412,-0.121273,-0.121273,-0.051402,-0.146476,-0.106984,-0.119617,-0.114514,-0.116922,-0.081917,-0.118276,-0.119617
2,-0.363603,0.652129,-0.645738,-0.316541,0.995764,-0.053629,-0.264139,-0.625611,0.490886,-1.185395,-0.152266,-0.10128,-0.121273,-0.078989,-0.087935,-0.136795,-0.085669,-0.113117,-0.069489,-0.097704,-0.080952,-0.096893,-0.111702,-0.16327,-0.059817,-0.141142,-0.117262,8.33683,-0.111346,-0.114861,-0.077989,-0.13262,-0.164268,-0.167475,-0.01969,-0.085209,-0.16352,-0.103215,-0.168451,-0.137967,-0.115896,-0.14785,-0.151905,-0.149755,-0.125481,-0.075434,-0.11658,-0.144252,-0.12675,-0.098108,-0.136501,-0.061744,-0.168936,-0.087486,-0.132317,-0.114166,-0.09851,-0.064225,-0.087035,-0.165013,-0.122582,-0.162267,-0.158196,-0.086582,-0.119283,-0.139995,-0.131099,-0.159988,-0.112412,-0.121273,-0.121273,-0.051402,-0.146476,-0.106984,-0.119617,-0.114514,-0.116922,-0.081917,-0.118276,-0.119617
3,-0.363603,-0.411379,-0.896599,0.039558,-0.873621,-0.053629,-0.264139,-0.625611,0.490886,-0.185945,-0.152266,-0.10128,-0.121273,-0.078989,-0.087935,-0.136795,-0.085669,-0.113117,-0.069489,-0.097704,-0.080952,-0.096893,-0.111702,-0.16327,-0.059817,-0.141142,-0.117262,-0.11995,-0.111346,-0.114861,-0.077989,-0.13262,-0.164268,-0.167475,-0.01969,-0.085209,-0.16352,-0.103215,-0.168451,-0.137967,-0.115896,-0.14785,-0.151905,-0.149755,-0.125481,-0.075434,-0.11658,-0.144252,-0.12675,10.192856,-0.136501,-0.061744,-0.168936,-0.087486,-0.132317,-0.114166,-0.09851,-0.064225,-0.087035,-0.165013,-0.122582,-0.162267,-0.158196,-0.086582,-0.119283,-0.139995,-0.131099,-0.159988,-0.112412,-0.121273,-0.121273,-0.051402,-0.146476,-0.106984,-0.119617,-0.114514,-0.116922,-0.081917,-0.118276,-0.119617
4,0.748320,1.006632,1.149900,-0.159814,0.995764,-0.053629,-

In [23]:
lm2=load(open('lm2.pkl','rb'))

In [24]:
final_answers=lm2.predict(transformed_holdout)

In [25]:
final_answers2=pd.Series(final_answers)

In [26]:
final_answers2

0       535379.884240
1       535379.884240
2       484577.384499
3       435757.759611
4       501683.464728
            ...      
4318    552529.643721
4319    428027.032412
4320    351566.450642
4321    456941.626474
4322    352091.419903
Length: 4323, dtype: float64

In [27]:
final_answers2.to_csv('housing_pred_ASFD.csv',header=None)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
